# Pull Polygons from TerraMatch API

This notebook sets up the process to pull polygon geometries and metadata from the TerraMatch API.

In [1]:
import yaml
import pandas as pd
from tm_api_utils import pull_tm_api_data, patch_tm_api_data
from tqdm import tqdm
import json
import sys
from datetime import datetime
sys.path.append('../src/')
import api_utils as api
import process_tm_api_results as clean

## Set file paths

In [2]:
# Naming convention
run_name = 'cohort2'
run_dir = 'tf_cohort2'

# Today's date
today = datetime.today().strftime('%Y-%m-%d') # Check computer date before running

# Files
approved_projects_file = '../projects_all_approved_202502211226.csv' # List of approved projects (infile)
tm_api_pull_results_file = f'../data/{run_dir}/tm_api_response_prod_{run_name}_{today}.json' # Save a JSON file that stores the results of the TM API pull; read it back in to clean the results (outfile, infile)
polygon_features_file = f'../data/{run_dir}/tm_api_{run_name}_{today}.csv' # Save the cleaned polygon features csv in the terrafund-portfolio-analyses repo (outfile)
polygon_features_maxar_file = f'/home/darby/github_repos/maxar-tools/data/{run_dir}/polygon_data/tm_api_{run_name}_{today}.csv' # Save the cleaned polygon features csv in the maxar-tools repo (outfile)

## Set up token and API URL

In [3]:
# Set up token access
auth_path = '../secrets.yaml'
with open(auth_path) as auth_file:
    auth = yaml.safe_load(auth_file)
headers = {
    'Authorization': f"Bearer {auth['access_token']}"
    }

In [4]:
# TerraMatch API URLs
staging_url = "https://api-staging.terramatch.org/research/v3/sitePolygons?" # Use for testing queries
prod_url = "https://api.terramatch.org/research/v3/sitePolygons?" # Use to pull data for analysis

## Create lists of projects to pull

In [5]:
# Read in list of approved projects (2025-02-21)
full = pd.read_csv(approved_projects_file)
full.head()

,project_id,project_name,cohort,organization_name,organisation_id,country,framework_key,description,status,project_status
0,c462918b-47f7-4ed5-99e0-7fec6e342036,"""Nakuru Eco-Reforestation Project""",terrafund-landscapes,NaN,1382,KE,enterprises,MAIN ACTIVITIES\nJAN - FEB 2024- TREE NURSERY...,approved,NaN
1,caae56f9-0bb6-45a2-9d77-ff088b085917,0726 project,NaN,NaN,6283,BR,terrafund,org 0726,approved,NaN
2,c004619e-c1aa-4f7f-b56b-c8f9b4385d4e,1,NaN,NaN,1582,AL,terrafund,1,approved,existing_expansion
3,ff7a1237-2fe4-4d91-a284-fdf71a2bb787,1220,ppc,NaN,6393,AU,ppc,NaN,approved,NaN
4,6083e1cf-a636-4c64-9253-ac86cd08f5d7,3SC Production 2.3,NaN,NaN,3279,AF,terrafund,3SC Production 2.3,approved,new_project


In [6]:
# Create lists of projects by Cohort (and split cohort 1 into projects within the TF landscapes and outside of the TF landscapes)
cohort1 = full[full['cohort'] == 'terrafund']
cohort1_landscapes = cohort1[cohort1['country'].isin(['BI', 'CD', 'RW', 'KE', 'GH'])]
cohort1_non_landscapes = cohort1[~cohort1['country'].isin(['BI', 'CD', 'RW', 'KE', 'GH'])]
cohort2 = full[full['cohort'] == 'terrafund-landscapes']

ppc = full[full['cohort'] == 'ppc']
ppc.head()

,project_id,project_name,cohort,organization_name,organisation_id,country,framework_key,description,status,project_status
3,ff7a1237-2fe4-4d91-a284-fdf71a2bb787,1220,ppc,NaN,6393,AU,ppc,NaN,approved,NaN
6,dc4d7f6a-ed16-470d-86af-44a04fffce50,Acción Andina,ppc,NaN,6,PE,ppc,NaN,approved,NaN
19,b399dd24-0330-4abd-b3b9-acfb30e511eb,AMBIO/Coopertiva AMBIO SC de RL/Restauración f...,ppc,NaN,562,MX,ppc,NaN,approved,NaN
32,c1020fec-836d-4196-a321-e1cdfa159dd2,Cafe Capitán/Restauración de paisaje en comuni...,ppc,NaN,562,MX,ppc,NaN,approved,NaN
33,1977b649-908c-46c3-836d-f4f6485427c2,CEPAN - Flagship,ppc,NaN,553,BR,ppc,NaN,approved,NaN


In [7]:
# Create a list of project ids to query
ids = list(set(cohort2.project_id))
len(ids)

# Create a short list of ids for testing 
ids = ids[2:4]
ids

['394e4de8-7b45-4834-a467-241ae4a16ece',
 '9bdf80ad-4c2d-4292-a21b-95b819f9c4be']

## Pull polygons from TM API

In [8]:
results = api.pull_wrapper(prod_url, headers, ids, outfile=tm_api_pull_results_file)

Pulling Projects: 100%|██████████| 2/2 [02:13<00:00, 66.84s/project]

Results saved to ../data/tf_cohort2/tm_api_response_prod_cohort2_2025-05-01.json


In [9]:
df = pd.DataFrame(results)

In [10]:
print(len(df))
print(f"df has {df.project_id.nunique()} unique projects")
print(f"df has {df.poly_id.nunique()} unique polygons")
df['project_id'].value_counts()
df.head()

35
df has 2 unique projects
df has 35 unique polygons


,status,plantStart,calcArea,plantEnd,practice,targetSys,distr,numTrees,name,siteId,projectId,indicators,siteName,geometry,establishmentTreeSpecies,reportingPeriods,lightResource,poly_id,project_id
0,approved,2024-06-07,4.841681,2024-07-31,tree-planting,natural-forest,full,4500,Prakaw Forest Reserve/ Tropenbos Ghana site,e7e9b053-6227-464a-991b-2313561a5dfb,394e4de8-7b45-4834-a467-241ae4a16ece,"[{'indicatorSlug': 'treeCover', 'yearOfAnalysi...",Prakaw Forest Reserve/Tropenbos Ghana Site,"{'type': 'Polygon', 'coordinates': [[[-1.02029...",[],"[{'dueAt': '2024-07-30T00:00:00.000Z', 'submit...",False,9a9328b7-b92b-4b6f-80eb-e7b91bc8ce64,394e4de8-7b45-4834-a467-241ae4a16ece
1,approved,2024-07-13,1.226562,2024-06-29,tree-planting,agroforest,partial,23,Osei Dickson,38822208-eef6-4481-8518-c3e54203d9b6,394e4de8-7b45-4834-a467-241ae4a16ece,"[{'indicatorSlug': 'treeCover', 'yearOfAnalysi...",Bankame/Tropenbos Ghana site,"{'type': 'Polygon', 'coordinates': [[[-1.22514...",[],"[{'dueAt': '2024-07-30T00:00:00.000Z', 'submit...",False,9e88e2a8-ed28-4e35-8b04-7fc6bb1ea9ba,394e4de8-7b45-4834-a467-241ae4a16ece
2,approved,2024-06-13,1.279760,2024-06-30,"assisted-natural-regeneration,tree-planting",agroforest,partial,24,Sule Mohammed,38822208-eef6-4481-8518-c3e54203d9b6,394e4de8-7b45-4834-a467-241ae4a16ece,"[{'indicatorSlug': 'treeCover', 'yearOfAnalysi...",Bankame/Tropenbos Ghana site,"{'type': 'Polygon', 'coordinates': [[[-1.22909...",[],"[{'dueAt': '2024-07-30T00:00:00.000Z', 'submit...",False,ff834801-abb9-4941-81ef-4804ed98a4b6,394e4de8-7b45-4834-a467-241ae4a16ece
3,approved,2024-06-12,0.408642,2024-07-31,"assisted-natural-regeneration,tree-planting",agroforest,partial,15,Dora Agyeiwaa,38822208-eef6-4481-8518-c3e54203d9b6,394e4de8-7b45-4834-a467-241ae4a16ece,"[{'indicatorSlug': 'treeCover', 'yearOfAnalysi...",Bankame/Tropenbos Ghana site,"{'type': 'Polygon', 'coordinates': [[[-1.22097...",[],"[{'dueAt': '2024-07-30T00:00:00.000Z', 'submit...",False,d83c48a0-07ab-4bb5-955a-d39c2b83b34d,394e4de8-7b45-4834-a467-241ae4a16ece
4,approved,2024-06-12,1.586834,2024-07-31,"assisted-natural-regeneration,tree-planting",agroforest,partial,31,Mercy Addo,38822208-eef6-4481-8518-c3e54203d9b6,394e4de8-7b45-4834-a467-241ae4a16ece,"[{'indicatorSlug': 'treeCover', 'yearOfAnalysi...",Bankame/Tropenbos Ghana site,"{'type': 'Polygon', 'coordinates': [[[-1.21646...",[],"[{'dueAt': '2024-07-30T00:00:00.000Z', 'submit...",False,1154b594-b8e6-4baa-8045-0a58c70c43fa,394e4de8-7b45-4834-a467-241ae4a16ece


## Clean Attributes

In [ ]:
# Load the saved JSON file
with open(tm_api_pull_results_file, 'r') as file:
    project_results = json.load(file)

In [ ]:
# Clean the csv and transform it into a dataframe
## Identifies and converts invalid plantstart and plantend dates to NaT
## Saves one copy of the polygon features csv to the terrafund-portfolio-analysis repo and one to the maxar-tools repo
clean_api = clean.process_tm_api_results(project_results,
                                         outfile1 = polygon_features_file,
                                         outfile2 = polygon_features_maxar_file)